In [1]:
# Imports
import os
import json
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
verbose = False #Set to true to see more output information

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

In [2]:
# user_command = f'''
# Generate a small program in IEC61131-3 that sends an email using following parameters:
# 1. To: "receiver@se.com"
# 2. Subject: "Test email"
# 3. Body: "This is a test email"
# 4. From: "sender@se.com"

# Authentication required.
# Message should be sent with high priority.
# Use Login to authenticate using "corrado" as user and "p@ssw0rd123" as password.
# Verify that the email has been sent successfully, if not print the error message.

# '''

In [3]:
# user_command = f'''
# Generate a small program in IEC61131-3 that receives an email using following parameters:
# 1. from: "bart_simpson@foo.com"
# 2. if received email contains "joke" send an email with following parameters:
#     1. To: "alert@foo.com"
#     2. Subject: "Alert email"
#     3. Body: "A joke email has been received"
#     4. From: "sender@se.com"
#     5. Authentication required.
#     6. Message should be sent with low priority.
#     7. No Authentication required.
#     8. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.
#     9. Consider all the case of failure, for example if the email server is not reachable,in this case 
# 3. else saves the content of the email to a file named "print_email.txt".


# '''

In [4]:
user_command = f'''
Generate a small program in IEC61131-3 that receives all the emails sent to the following email address: "bart_simpson@foo.com" then for each email received, do the following:
2. if received email contains "spam" send an email with following parameters:
    1. To: "spam@foo.com"
    2. Subject: "Alert spam email"
    3. Body: "A spam email has been received"
    4. From: "guardian@se.com"
    5. No Authentication required.
    6. Message should be sent with low priority.    
    7. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.    
3. else saves the content of the email to a file named "print_email.txt".


'''

In [5]:
rag_content='''
EMailHandling Library Guide
The EmailHandling library provides email client functions that allow your controller to send emails to one or several recipients with the possibility to customize the content. The protocol type used is SMTP as standard for email traffic. It is also possible to receive or delete emails from a server using the Post Office Protocol 3 (POP3).
This library supports SMTP and POP3 via a secured connection using TLS.
Whether a connection using TLS is supported depends on the controller where the FB_TcpClient2 is used. Refer to the specific manual of your controller to verify if TCP communication using TLS is supported.
You can connect your controller to an email server to send emails that inform about your machine status or report on key performance indicators.
Library title: EmailHandling
Category: Communication
Component: Internet Protocol Suite
Default namespace: SE_Email
For this library, qualified-access-only is set. This means, that the POUs, data structures, enumerations, and constants have to be accessed using the namespace of the library. The default namespace of the library is SE_EMail.

General Considerations
Consider the following limitations for email transfer:
 Only ASCII symbols are supported.
 Only IPv4 IP addresses are supported.
 The EmailHandling incorporates pointers on addresses.
 Receive acknowledgement is not supported.
 Sending or receiving files via email leads to loss of file attributes
 Archiving emails (sent and received items) has to be performed in the application program.
Automatic storage of email traffic on the controller file system is not implemented because
emails are mainly handled in the controller RAM (Random Access Memory).
ET_AuthenticationMode
The enumeration ET_AuthenticationMode defines the mode of authentication that is required to log into an email server.
Enumeration Elements
Name, Data Type, Value,Description
NoAuthentication, INT,0,No user authentication required
Login, INT,LOGIN mechanism used for authentication
Plain, INT, PLAIN mechanism used for authentication.

Used By
 FB_SendEMail
ET_Command
The enumeration ET_Command indicates the command that is executed.
Enumeration Elements
Name, Data Type, Value,Description
NoCommand,INT,0,No command is executed.
CheckInbox,INT,1,The number of emails available on the server is verified.
Receive,INT,2,The emails are received from the server.
Delete,INT,3,The email specified with the unique ID at the input i_sUniqueID is deleted from the server.
Used By
 FB_Pop3EMailClient
ET_EMailStatus
The enumeration ET_EMailStatus provides status information of the email.
Enumeration Elements
Name, Data Type, Value,Description
Empty INT 0 The structure is empty or the email data is not valid.
Ok INT 1 The email has been successfully received
SenderBlocked INT 2 The sender address is blocked by the white list, or is not found in the header of the email, or exceeds the maximum length of 200 bytes.
NotSupported, INT,3, The format of the email is not supported.,Supported Content-Types (not valid for the file,attachment part): text/plain, multipart/mixed. Supported Content-Transfer Encoding schemes: 7-bit, quoted-printable, base 64.
InvalidHeader,INT,4,The header of the email is invalid.
InvalidAttachmentPath,INT,5,The path to the attachment of the email is invalid:  The path exceeds the maximum length of 255 bytes, or No file name is provided.
InvalidAttachmentExtension,INT,6,The attachment of the email has an invalid extension. It is not possible to receive an attachment with one of the following extensions: .ap_, .app, .cf_,.cfg,.crc,.err,.frc,.log,.prj,.rcp,.rsi,.urf
Used By
 FB_Pop3EMailClient
ET_Priority
The enumeration ET_Priority defines the priority level that is assigned to the email.
Enumeration Elements
Name, Data Type, Value,Description
Normal,INT, 0-3,The email priority level is set to normal.
VeryHigh,INT,1,The email priority level is set to very high.
High,INT,2,The email priority level is set to high.
Low,INT,4,The email priority level is set to low.
VeryLow,INT,5,The email priority level is set to very low.
Used By
 FB_SendEMail
ET_Protocol
The enumeration ET_Protocol defines the type of protocol that is used for email transfer.
Name, Data Type, Value,Description
SMTP,INT,0,SMTP (Simple Mail Transfer Protocol) is used for email transfer.
eSMTP,INT,1, eSMTP (extended Simple Mail Transfer Protocol) is used for email transfer. Select this protocol to use additional protocol extensions, for example authentication,attachment, or email priority.
Used By
 FB_SendEMail

ET_Result
The enumeration ET_Result contains the possible values that indicate the result of operations executed by the function block.
If q_xError of a function is FALSE, one of the following status messages is shown.

Enumeration Elements
Name, Data Type, Value,Description
Ok,UDINT,0,The operation completed successfully.
Disabled,UDINT,1,The function block is disabled.
Initializing,UDINT,2,The function block is initializing.
Ready,UDINT,4,The function block is ready.
ConnectingToServer,UDINT,5,The client is connecting to the server.
DisconnectingFromServer,UDINT,6,The client is disconnecting from the server.
NoCommand,UDINT,7,No command selected.
SendingEMail,UDINT, 8,An email is being sent to the server.
ConnectedToServer,UDINT,50,A connection to the server has been established.
MessageFromServerReceived,UDINT,52,Data exchange with server, A message has been received.
MessageToServerSent,UDINT,54,Data exchange with server: A message has been sent to the server.
SendingMessageToServer,UDINT,55,Data exchange with server: A message is being sent to the server.
Busy,UDINT,56,The function block is busy.
WaitingForExpectedMessage,UDINT,58,Data exchange with server: Waiting for expected message to continue.
OpeningAttachment,UDINT,60,The attachment is opened on the file system of the controller.
SendingAttachment,UDINT,62,The process of sending the attachment is in progress.
DownloadingEMail,UDINT,64,The email is downloaded from the server.
SavingAttachment,UDINT,66,The process of saving the attachment in the file system of the controller is in progress.
DeletingEMail,UDINT,68,The email is deleted from the server.
ServerOk,UDINT,70,The server is in operational state.
DeletingFile,UDINT,72,The invalid file is deleted from the system.
InvalidAuthenticationMode,UDINT,101,It has been detected that the value of i_etAuthenticationMode is not valid. 
InvalidServerIP,UDINT,102,It has been detected that i_sServerIP is empty. This is not valid.
InvalidDomainName,UDINT,104,It has been detected that i_sDomainName is empty. This is not valid.
InvalidSenderEMail,UDINT,106,It has been detected that i_sSenderEMail is empty. This is not valid.
InvalidRecipientEMail,UDINT,108,It has been detected that i_sRecipientEMail is empty. This is not valid.
InvalidProtocol,UDINT,110,It has been detected that the value of i_etProtocol is not valid.
InvalidMessageAddress,UDINT,111,It has been detected that the value of i_pbyMessage is not valid.
AuthenticationFailed,UDINT,112,Authentication has not been successful: Username (i_sUsername) or password (i_sPassword) are not correct.
AuthenticationRequired,UDINT,113,Authentication is required: Enable an authentication mode.
InvalidUsername,UDINT,114,It has been detected that i_sUsername is empty. This is not valid.
InvalidPassword,UDINT,116, It has been detected that i_sPassword is empty. This is not valid.
FunctionNotSupported,UDINT,118,The selected function, such as the selected authentication mode, is not supported. Set i_etProtocol to ET_Protocol.eSMTP or try to use another i_etAuthenticationMode.
SyntaxError,UDINT,120,A syntax error, such as empty recipient address, has been detected.
RecipientAddressTooLong UDINT 122 It has been detected that the recipient address exceeds the allowed length of 200 bytes.
MailboxUnavailable,UDINT,124,The requested action has not been performed because the mailbox is not available.
OpenAttachmentFailed,UDINT,126,Unable to open the attachment.
ReadAttachmentFailed,UDINT,128,Unable to read the attachment.
CloseAttachmentFailed,UDINT,130,Unable to close the attachment.
MessageRejectedFromRecipient,UDINT,132,The email has been rejected by the recipient.
EMailNotFound,UDINT,133,Corresponding email has not been found on the server. Verify the value of i_sUniqueId.
InvalidPriority,UDINT,134,The value of i_etPriority is invalid.
InvalidNumberEMailsToReceive,UDINT,135,It has been detected that the value of i_uiEMailsToReceive is not valid. It must be greater than 0 and less or equal to GPL.Gc_uiInboxSize.
ServerError,UDINT,136,The server has detected an error.
InvalidWhiteListSize,UDINT,137,It has been detected that the value of i_udiWhiteListSize is not valid. It must be greater than 0.
InvalidFilePath,UDINT,138,It has been detected that the value of i_sFilePath is not valid.
InvalidMaxNumberOfAttachments,UDINT,139,The value of GPL.Gc_udiMaxNumberOfAttachments is not valid. It must be greater than 0.
InvalidMailboxBuffer,UDINT,140,The value of i_pbyMailboxBuffer is not valid.
InvalidCommand,UDINT,141,The value of i_etCommand is not valid
InvalidBufferSize,UDINT,142,The value of i_udiBufferSize is not valid. It must be greater than 0.
NotSupported,UDINT,143,This function is not supported.
InvalidInboxSize,UDINT,144,The value of Gc_udiInboxSize is not valid. It must be greater than 0.
InvalidUniqueId,UDINT,145,The value of i_sUniqueID is not valid.
SaveAttachmentFailed,UDINT,146,The attempt to save the attachment to the file system of the controller was not successful.
InvalidWhiteList,UDINT,147,The value of i_pyWhiteListSender is not valid. As i_udiWhiteListSize is greater than 0,i_pyWhiteListSender must have a valid address (>0).
MailboxBufferFull,UDINT,148,The buffer of the mailbox is full.The number of received bytes is greater than i_udiBufferSize.
DecodingError,UDINT,149,An internal error has been detected while decoding an email.
FailedToConnectToServer,UDINT,150,The connection to the server has not been established. Verify the Ethernet connection or the parameters i_sServerIP /i_sServerPort.
InvalidMessageFromServer,UDINT,152,It has been detected that the message received from the server is invalid.
UnexpectedMessageFromServer,UDINT,153,An unexpected message has been received from the server.
FailedToReceiveMessageFromServer,UDINT,154,A timeout has expired: No answer has been received from the server within the given time.
FailedToSendMessageToServer,UDINT,156,A communication error has been detected: Unable to send message to server.
UnexpectedProgramBehavior,UDINT,200,An internal error has been detected.
Used By
FB_SendEMail
FB_PopEMailClient
ST_CredentialsSendEMail
Description
The structure ST_CredentialsSendEMail contains the user-specific information for connecting to an external email server for sending emails.
Structure Element
Name,Data type,Description
i_sServerIP,STRING[GPL.Gc_uiIpStringSize],The IP address of the external email server.
i_uiServerPort,UINT,The port of the external email server.
i_etProtocol,ET_Protocol,Enumeration which indicates the protocol.
i_sDomainName,STRING[200],The domain of the client.
i_sSenderEMail,STRING[200],The email address of the sender.
i_sUsername,STRING[60],The username to access the external email server.
i_sPassword,STRING[60],The password to access the external email server.i_etAuthenticationMode,ET_AuthenticationMode,Enumeration which indicates the authentication mode.
stTlsSettings, ST_TlsSettings,Structure to specify the TLS (Transport Layer Security) configuration.

Used By
FB_SendEMail

ST_CredentialsReceiveEMail
Description
The structure ST_CredentialsReceiveEMail contains the user-specific information for connecting to an external email server for receiving and deleting emails using POP3.
Structure Element
Name,Data type,Description
i_sServerIP,STRING[GPL.Gc_uiIpStringSize],The IP address of the external email server.
i_uiServerPort,UINT,The port of the external email server.
i_sUsername STRING[200] The username to access the external email server.
i_sPassword,STRING[60],The password to access the external email server.
i_pbyWhiteListSender,POINTER TO BYTE,The start address of the string containing the address(es) of the white list.If this list contains more than one entry, the email addresses must be separated by a semicolon. The aximum size of a single address is restricted to 200 bytes. An empty string blocks all emails. Enter an asterisk in  combination with a domain (*@yourdomain.com) to allow receiving emails from senders of this domain. Emails from other domains are blocked.
i_udiWhiteListSize,UDINT,Size of the white list.
i_xOverwriteAttachment,BOOL,If TRUE, attachments stored in the EMailAttachments folder are overwritten if an attachment with the same name is downloaded from the server. If FALSE, the receive process is aborted as soon as a second attachment with identical name is to be downloaded. The output q_sResultMsg indicates the file name that generated the error.
stTlsSettings,ST_TlsSettings, Structure to specify the TLS (Transport Layer Security) configuration.
Used By
FB_Pop3EMailClient
ST_Email
Description
The structure ST_EMail contains the information of a received email.
Structure Element
Name,Data type,Description
q_etEMailStatus,ET_EMailStatus,Status of the email.
q_pbyDate,POINTER TO BYTE,The start address of the string containing the date.
q_udiLengthOfDate,UDINT,Length of the date string.
q_pbySenderEMail,POINTER TO BYTE,The start address of the string containing the sender email address.
q_udiLengthOfSenderEMail,UDINT,Length of the sender email address string.
q_pbySubject,POINTER TO BYTE,The start address of the string containing the subject of the email.
udiLengthOfSubject,UDINT,Length of the subject string.
q_pbyMessage,POINTER TO BYTE,The start address of the string containing the email message.
q_udiLengthOfMessage,UDINT,Length of the message string.
q_asAttachmentPath,ARRAY[1..GPL.Gc_udiMaxNumberOfAttachments] OF STRING[255],Relative or absolute path(s) to the attachment file(s) on the file system.
q_audiSizeOfAttachment,ARRAY[1..GPL.Gc_udiMaxNumberOfAttachments] OF UDINT,Size(s) of the attachment file(s) on the file system.
q_sUnique,ID STRING[70],Unique ID of the email.
ST_TlsSettings
Description
The structure ST_TlsSettings contains the parameter used for the connection to the server and contains the TLS (Transport Layer Security)
Element,Data type,Default value,Description
xUseTls,BOOL,FALSE,If TRUE, the connection is established using TLS.
xSendClientCert,BOOL,FALSE,Set to TRUE if a client certificate is required.
sCertName,STRING[255],-,Specifies the client certificate file. If the value is a null string, the default certificate of the controller is used. The name corresponds to the value “Issued for” from the controller certificate. The name can be obtained from the certificate using the Security Screen in Logic Builder.
etProtocolTCPUDP.ET_TlsPrototocol,TCPUDP.ET_TlsProtocol.TLSv12,Protocol used for secured connection. etCertVerifyMode,TCPUDP.ET_CertVerifyMode, TCPUDP.ET_CertVerifyMode.TrustedOnly, Mode for the certificate verification.

Global Parameter List (GPL)
Description
The global parameter list contains the global parameters of the EmailHandling library. They can be overwritten specifically for your project in the Library Manager.
Variable,Data type,Default value,Description
Gc_udiRecipientEMailSize,UDINT,200,Maximum number of bytes of i_sRecipientEMail
Gc_tTimeOut,TIME,TIME#10m0s0ms,Maximum time to receive an answer from the server.
Gc_uiIpStringSize,UINT,15,Maximum size for IP addresses, such as i_sServerIP.
Gc_udiReceiveBufferSize,UDINT,1000,Maximum size of the receive buffer that is required to communicate with the server.
Gc_xUseControllerTime,BOOL,FALSE,If TRUE, the controller time is used for creating the time stamp indicating when the email was sent.
Gc_udiInboxSize,UDINT,10,Maximum size of the inbox. This value determines the maximum number of mails that can be downloaded withcone receive command.
Gc_udiMaxNumberOfAttachmentPaths,UDINT,1,Maximum size of the array ST_EMail.asAttachmentPath. It contains absolute or relative path(s) on the file system of the controller where downloaded attachments are stored. This variable does not limit the number of downloaded attachments.
Gc_xUseTop,BOOL,TRUE, If TRUE internally the TOP0 command is used to download only the header of the email. This is useful if the white list feature is used to verify the email address of the sender before the entire message is downloaded. If the TOP command is not supported by your email server (indicated by q_sResultMsg), set this variable to FALSE.
Gc_xUseUniqueId,BOOL,TRUE, If TRUE the UIDL command is used to retrieve the unique ID of the email. If the UIDL command is not supported by your email server (indicated by q_sResultMsg), set this variable to FALSE.

Global Variable List (GVL)
Description
The global variables list contains the global variables of the library.
Variable,Data type,Default value,Description
Gc_xOnlineChangeAllowed,BOOL,FALSE, If TRUE the online change detection is successfully initialized. Execute an online change while the POU is busy only if Gc_xOnlineChangeAllowed =TRUE.

FB_SendEMail

Task
The FB_SendEMail function block includes the related functions for sending emails. Each instance handles one SMTP connection.
Functional Description
The FB_SendEMail function block is the user-interface to interact with an external email server. It allows you to send emails.
After the function block has been enabled and is being executed, a TCP connection to the email server is established using the user credentials that have been submitted using iq_stCredentials. As soon as the connection has been established, email data is transferred to the server. When the data transfer is completed, the TCP connection to the email server is automatically closed by the function block.
When executing the function block, the input i_pbyMessage is stored internally for further use. In case an online change event is detected while the function block is executed (q_xBusy = TRUE), the internally used variables are updated with the present value of the input.
NOTE: Do not reassign the i_pbyMessage to a different memory area while the function block is executed.
As long as the function block is executed, the output q_xBusy is set to TRUE. The output q_xDone is set to TRUE after the function block has been executed successfully.
The diagram shows the signal behavior of the inputs and outputs of the function block:
Status messages and diagnostic information are provided using the outputs q_xError (TRUE if an error has been detected), q_etResult, and q_etResultMsg.
To acknowledge detected errors, disable and re-enable the function block to be able to resend an email.
Interface
Input,Data type,Description
i_xEnable,BOOL,Activation and initialization of the function block.
i_xExecute,BOOL,The function block sends an email upon rising edge of this input.
i_sRecipientEMail,STRING[GPL.Gc_udiRecipientEMailSize],The string containing the recipient email address(es).
i_sSubject,STRING[200],Subject of the email.
i_pbyMessage,POINTER TO BYTE,Start address of the string in which the message is stored.
i_udiSizeOfMessage,UDINT,Size of message data.
i_etPriority,ET_Priority,The enumeration indicating the priority level that is assigned to the email.
i_sAttachmentPath,STRING[255],Absolute or relative path to the attachment located on the controller file system. If this string is empty, no attachment is sent.
i_sCcRecipientEMail,STRING,[GPL.Gc_udiRecipientEMailSize],The string containing the recipient email address(es) assigned to the CC field.
i_sBccRecipientEMail,STRING[GPL.Gc_udiRecipientEMailSize],The string containing the recipient email address(es) assigned to the BCC field.
Input / Output,Data type,Description
iq_stCredentials,ST_CredentialsSendEMail,Used to pass the structure containing user settings, such as user name or password.
Output,Data type,Description
q_xActive BOOL,If the function block is active, this output is set to TRUE.
q_xReady,BOOL,If the initialization is successful, this output signals a TRUE as long as the function block is capable of accepting inputs.
q_xBusy,BOOL If this output is set to TRUE the function block execution is in progress.
q_xDone,BOOL,If this output is set to TRUE, the execution has been completed successfully.
q_xError,BOOL,If this output is set to TRUE an error has been detected. For details, refer to q_etResult and q_etResultMsg.
q_etResult, ET_Result, Provides diagnostic and status information
q_sResultMsg,STRING[80],Provides additional diagnostic and status information.

FB_Pop3EMailClient
Task
The FB_Pop3EMailClient function block includes the related functions for receiving and deleting emails using POP3. Each instance handles one POP3 connection.
Functional Description
The FB_Pop3EMailClient function block is the user-interface to interact with an external POP3 (email) server. It allows you to receive and delete emails. By using attachments of received emails you are able to get input for several system features which are based on files located on the system memory. Certain file extensions are not allowed to be stored on the controller file system via FB_Pop3EMailClient (refer to the ET_EmailStatus.InvalidAttachmentExtension parameter). This applies to files that are handled automatically by the controller and to system files, such as the controller firmware to help to prevent unintended overwriting.

After the function block has been enabled and is being executed, a TCP connection to the POP3 server is established using the user credentials that have been submitted using iq_stCredentials. As soon as the connection has been established, the command specified with i_etCommand is executed.
When executing the function block, the pointers at i_pbyMessage and iq_stCredentilas.i_pbyWhiteListSender are stored internally for further use. In case an online change event is detected while the function block is executed (q_xBusy = TRUE), the internally used variables are updated with the present value of the pointers.
NOTE: Do not reassign the i_pbyMessage and iq_stCredentilas.i_pbyWhiteListSender to a different memory area while the function block is executed.
When the data transfer is completed, the TCP connection to the POP3 server is closed by the function block. Received emails are deleted from the POP3 server.
The function block FB_Pop3EMailClient saves the information of received emails in the memory area addressed by the pointer i_pbyMailboxBuffer. The positions inside the buffer, which contain the respective information of the several emails, is indicated by the pointers inside the structure iq_astInbox. In case the function block FB_Pop3EMailClient is enabled and an online change event is detected, the function block recognizes a possible change of the pointer at i_pbyMailboxBuffer and so the pointers provided within the structure iq_astInbox are updated accordingly.
NOTE: Process the information from the received emails before modifying the address of the pointer i_pbyMailboxBuffer for the next execution.
You can manually delete emails by specifying the email with the unique ID at the input i_sUniqueId and executing the delete command with i_etCommand. By executing further commands, the inbox structure available at q_astInbox containing the references to the email data are reset.
Received emails are held in volatile memory. The volatile memory is cleared when power is removed, and all held emails are therefore deleted.
As long as the function block is executed, the output q_xBusy is set to TRUE. The output q_xDone is set to TRUE after the function block has been executed successfully.
The diagram shows the signal behavior of the inputs and outputs of the function block.
Status messages and diagnostic information are provided using the outputs q_xError (TRUE if an error has been detected), q_etResult, and q_etResultMsg.
To acknowledge detected errors, disable and re-enable the function block to be able to receive or delete an email.
Interface
Input,Data type,Description
i_xEnable,BOOL,Activation and initialization of the function block.
i_xExecute,BOOL,The function block receives or deletes an email upon rising edge of this input.
i_etCommand,ET_Command,The enumeration indicating the command to be executed.
i_pbyMailboxBuffer,POINTER TO BYTE, Start address of the first byte in which the incoming emails are stored.
i_udiBufferSize,UDINT,Size of the mailbox buffer.<
i_uiEMailsToReceive,UINT,Number of emails to receive from the server.
i_sFilePath,STRING[200],Path to the folder in the controller file system where the folder EMailAttachments is created. Inside this folder, the attachments of the received emails are stored. The file extension defined with the ET_EMailStatus.InvalidAttachmentExtension parameter cannot be stored. NOTE: If you receive a second attachment with identical name as an already available attachment in this folder, the older file may be overwritten if the global parameter ST_CredentialsReceiveEMail.i_xOverwriteAttachment is set to TRUE.
If this string is empty, the folder EMailAttachments is created at the default file path of the controller.
i_sUniqueID,STRING[70],The unique ID that is required to delete an email.After the email has been received from the server, the unique ID is displayed at the output q_astInbox.
Input / Output,Data type,Description
iq_stCredentials,ST_CredentialsReceiveEMail,Used to pass the structure containing user settings, such as user name or password.
iq_astInbox,ARRAY [1…GPL.Gc_udiInboxSize] OF ST_Email,Structure which contains the information of received emails.
Output,Data type,Description
q_xActive,BOOL,If the function block is active, this output is set to TRUE.
q_xReady,BOOL,If the initialization is successful, this output signals a TRUE as long as the function block is operational.
q_xBusy,BOOL,If this output is set to TRUE the function block execution is in progress.
q_xDone,BOOL,If this output is set to TRUE the execution has been completed successfully.
q_xError,BOOL,If this output is set to TRUE, an error has been detected. For details refer to q_etResult and q_etResultMsg.
q_etResult,ET_Result,Provides diagnostic and status information.
q_sResultMsg,STRING[80],Provides additional diagnostic and status information.
q_udiNumberOfEmails,UDINT,Depends on the executed i_etCommand: ET_Command.CheckInbox: Indicates the number of emails available on the server. ET_Command.Receive: Indicates the number of emails received from the server.If an error has been detected, this output provides the number of emails downloaded successfully. ET_Command.Delete: Indicates the number of emails deleted.
FC_EtResultToString
Task
Convert an enumeration element of type ET_Result to a variable of type STRING.
Functional Description
Using the function FC_EtResultToString, you can convert an enumeration element of type ET_Result to a variable of type STRING
Interface
Input,Data type,Description
i_etResult,ET_Result,Enumeration with the result
Return Value
Data type,Description
STRING(80),The ET_Result converted to text.If i_etResult is indeterminable the return value is: Unknown Result: <Value of the input i_etResult>

'''

In [6]:
libraries='''
Library title,Company,Category,Component,Description
CollisionDetection,Schneider Electric,Application,PacDrive Robotics,This library offers a set of functions to perform a collision check between two or more objects and a distance calculation between two or more objects. For more information, refer to the CollisionDetection Library Guide.
EMailHandling,Schneider Electric,Communication,Internet Protocol Suite, This library supports the implementation of an email client and provides function blocks for sending SMTP (Simple Mail Transfer Protocol) and receiving POP3 (Post Office Protocol) emails.NOTE: The communication is implemented using the TcpUdpCommunication library.AlarmManager,intern,Intern > AlarmManager,Core Repository,This library contains the components for the alarm handling.
CAA File,CAA Technical Workgroup,System,Core Repository,This library provides function blocks for accessing file directory systems and files.

'''
library_system_prompt=f'''
You are an assistant able to select a library based on input requirements.
Your goal is to return the library names selecting from the one provided in the context.
Favor the use of external libraries over internal libraries.
Return only the name of the libraries that matches the user input separated by a comma, if name of the library starts with CAA then append "from CodeSys" to the name of the library.

CONTEXT:
```
{libraries}
```
'''

In [7]:
messages=[]
messages.append({'role': 'system', 'content': library_system_prompt})
messages.append({'role': 'user', 'content': user_command})   

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

target_libraries= openai_response.choices[0].message.content
print(target_libraries)

EMailHandling, CAA File from CodeSys


In [8]:
# system message
system_message = f'''
You are an assistant with knowledge of the following topics:
1. IEC61131-3 languages
2. Structured Text
3. Function Block Diagram
4. IEC61131-3 coding standards
5. IEC61131-3 best practices
6. IEC61131-3 coding guidelines
7. IEC61131-3 programming
8. IEC61131-3 programming languages
9. Schneider Electric EcoStruxure Control Expert
10. Schneider Electric EcoStruxure Machine Expert
11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

Your job is to generate small examples of code using exclusively IEC61131-3 Structured Text base on user input.
You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

The code should make use of the following {target_libraries} libraries.

Use the following pieces of retrieved context to understand how to handle library related code.
CONTEXT:
```
{rag_content}
```
'''

In [9]:
messages=[]
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': user_command})  

token_count=count_tokens(rag_content);
print (f'Total user tokens: {token_count}')     

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
result= openai_response.choices[0].message

print (result.content)

Total user tokens: 6231
Sure, here is a small program that fulfills your requirements. This program is written in Structured Text, which is one of the IEC61131-3 languages.

```iecst
PROGRAM EmailHandler
VAR
    Pop3Client: FB_Pop3EMailClient;
    SendEmail: FB_SendEMail;
    EmailCredentials: ST_CredentialsSendEMail;
    ReceiveCredentials: ST_CredentialsReceiveEMail;
    Email: ST_Email;
    RetryCount: INT;
    FileHandler: CAA_File.TF_File;
    FileWriteResult: CAA_File.RTS_IEC_RESULT;
END_VAR

METHOD Run
    ReceiveCredentials.i_sUsername := 'bart_simpson@foo.com';
    ReceiveCredentials.i_sPassword := 'password'; // Replace with actual password
    ReceiveCredentials.i_sServerIP := 'pop3.foo.com'; // Replace with actual server IP
    ReceiveCredentials.i_uiServerPort := 110; // Replace with actual server port

    Pop3Client.iq_stCredentials := ReceiveCredentials;
    Pop3Client.i_xEnable := TRUE;
    Pop3Client.i_xExecute := TRUE;
    Pop3Client.i_etCommand := ET_Command.Receive